## Part II: `Candidate` Extraction

In [1]:
# If necessary:
import os
os.remove('snorkel.db');
os.system('cp snorkel.db\ corpus snorkel.db');

In [2]:
%load_ext autoreload
%autoreload 2

from snorkel import SnorkelSession
session = SnorkelSession()

## Loading the Training `Corpus`

First, we will load the `Corpus` that we preprocessed in Part I:

In [3]:
from snorkel.models import Corpus
from snorkel.utils import get_ORM_instance

corpus = get_ORM_instance(Corpus, session, 'Hardware Training')
print "%s contains %d Documents" % (corpus, len(corpus))

Corpus (Hardware Training) contains 80 Documents


## Defining a `Candidate` Schema

In [4]:
from snorkel.models import candidate_subclass

Part_Temp = candidate_subclass('Part_Temp', ['part','temp'])

## Writing a basic `CandidateExtractor`

In [5]:
from hardware_utils import OmniNgramsPart, OmniNgramsTemp

part_ngrams = OmniNgramsPart(n_max=3, split_tokens=None)
temp_ngrams = OmniNgramsTemp(n_max=3, split_tokens=None)

In [6]:
from hardware_utils import load_extended_parts_dict
gold_file ='data/hardware/hardware_gold.csv'
parts_dict = load_extended_parts_dict(gold_file)
print "Loaded %d part numbers." % len(parts_dict)

Loaded 1286 part numbers.


In [7]:
from snorkel.matchers import DictionaryMatch, RegexMatchSpan, Union

eeca_matcher = RegexMatchSpan(rgx='([b]{1}[abcdefklnpqruyz]{1}[\swxyz]?[0-9]{3,4}[\s]?[A-Z]{0,2}[0-9]?([-][A-Z0-9]{1,3})?)')
jedec_matcher = RegexMatchSpan(rgx='([123]N\d{3,4}[A-Z]?)')
jis_matcher = RegexMatchSpan(rgx='(2S[abcdefghjkmqrstvz]{1}[\d]{2,4})')
others_matcher = RegexMatchSpan(rgx='((NSVBC|SMBT|MJ|MJE|MPS|MRF|RCA|TIP|ZTX|ZT|TIS|TIPL|DTC|MMBT|PZT){1}[\d]{2,4}[A-Z]{0,3}([-][A-Z0-9]{0,3})?)')
parts_matcher = Union(eeca_matcher, jedec_matcher, jis_matcher, others_matcher)
# parts_matcher = DictionaryMatch(d=parts_dict)

temp_matcher = RegexMatchSpan(rgx=r'-[5-7][05]')

In [8]:
from snorkel.candidates import CandidateExtractor

ce = CandidateExtractor(Part_Temp, [part_ngrams, temp_ngrams], [parts_matcher, temp_matcher])

## Running the `CandidateExtractor`

In [9]:
%time train = ce.extract(corpus.documents, 'Hardware Training Candidates', session)
print "%s contains %d Candidates" % (train, len(train))

[========================================] 100%

CPU times: user 1min 12s, sys: 1.38 s, total: 1min 13s
Wall time: 1min 16s
Candidate Set (Hardware Training Candidates) contains 11549 Candidates


In [10]:
for c in train[:3]:
    print c

Part_Temp(Span("BC807K", parent=27872, chars=[94,99], words=[5,5]), ImplicitSpan("-55", parent=87287, words=[0,0], position=[0]))
Part_Temp(Span("BC817K", parent=27857, chars=[0,5], words=[0,0]), ImplicitSpan("-55", parent=87287, words=[0,0], position=[0]))
Part_Temp(Span("BC818K", parent=2149, chars=[9,14], words=[2,2]), ImplicitSpan("-55", parent=87287, words=[0,0], position=[0]))


### Saving the extracted candidates

In [11]:
session.add(train)
session.commit()

### Reloading the candidates

In [12]:
from snorkel.models import CandidateSet
from snorkel.utils import get_ORM_instance

train = get_ORM_instance(CandidateSet, session, 'Hardware Training Candidates')
print "%s contains %d Candidates" % (train, len(train))

Candidate Set (Hardware Training Candidates) contains 11549 Candidates


### Repeating for development and test corpora

In [13]:
for corpus_name in ['Hardware Development']:
    corpus = get_ORM_instance(Corpus, session, corpus_name)
    print "Extracting Candidates from %s" % corpus
    %time candidates = ce.extract(corpus.documents, corpus_name + ' Candidates', session)
    session.add(candidates)
    print "%s contains %d Candidates" % (candidates, len(candidates))
session.commit()

Extracting Candidates from Corpus (Hardware Development)
[========================================] 100%

CPU times: user 16.1 s, sys: 302 ms, total: 16.4 s
Wall time: 16.8 s
Candidate Set (Hardware Development Candidates) contains 3163 Candidates


In [14]:
# train = get_ORM_instance(Corpus, session, 'Hardware Training')
# dev = get_ORM_instance(Corpus, session, 'Hardware Development')
# test = get_ORM_instance(Corpus, session, 'Hardware Test')
# trainies = [d.name for d in train.documents]
# len(trainies)
# for d in test.documents:
#     if d.name in trainies:
#         print 'YES!'
# # for d in test.documents[:10]: print d

## TEMPORARY - Assessing Total Recall

In [15]:
from hardware_utils import entity_level_total_recall
from snorkel.utils import get_ORM_instance

train = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Training Candidates').one()
dev = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Development Candidates').one()
total_set = set([])
for c in train:
    total_set.add(c)
for c in dev:
    total_set.add(c)

In [16]:
print len(total_set)

14712


In [17]:
# import os
gold_file = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
(tp, fp, fn) = entity_level_total_recall(total_set, gold_file, relation=True)

Preparing gold set...
Preparing candidates...
[========================================] 100%

Scoring on Entity-Level Total Recall
Entity-level Candidates extracted: 1632 
Entity-level Gold: 544
Intersection Candidates: 424
----------------------------------------
Overlap with Gold:  0.78



In [18]:
# from snorkel.utils import ProgressBar
# target = sorted(list(fn))[-1]
# print target
# print "-------------------------------"
# pb = ProgressBar(len(candidates))
# for i, c in enumerate(list(candidates)[:]):
#     pb.bar(i)
#     if (c.part.parent.document.name.upper() == target[0].upper())
#         and c.part.get_span().upper() == target[1].upper()):
#         print c
# pb.close()
print "-------------------------------"
for c in sorted(list(fn))[:]:
    print c

-------------------------------
('BC546', 'BC548A', '-65')
('BC547', 'BC548A', '-65')
('BC547', 'BC549A', '-65')
('BC547', 'BC550A', '-65')
('BC547', 'BC550B', '-65')
('BC550', 'BC546A', '-65')
('BC550', 'BC546B', '-65')
('BC550', 'BC546C', '-65')
('BC550', 'BC547A', '-65')
('BC550', 'BC547B', '-65')
('BC550', 'BC547C', '-65')
('BC550', 'BC548A', '-65')
('BC550', 'BC548B', '-65')
('BC550', 'BC548C', '-65')
('BC550', 'BC549A', '-65')
('BC550', 'BC549B', '-65')
('BC550', 'BC549C', '-65')
('BC550', 'BC550A', '-65')
('BC550', 'BC550B', '-65')
('BC550', 'BC550C', '-65')
('BC818', 'BC817-16', '-65')
('BC818', 'BC817-25', '-65')
('BC818', 'BC817-40', '-65')
('BC818', 'BC818-16', '-65')
('BC818', 'BC818-25', '-65')
('BC818', 'BC818-40', '-65')
('CSEMS05382-1', 'BC546A', '-65')
('FAIRS19194-1', 'BC856A', '-65')
('FAIRS19194-1', 'BC856B', '-65')
('FAIRS19194-1', 'BC856C', '-65')
('FAIRS19194-1', 'BC857A', '-65')
('FAIRS19194-1', 'BC857B', '-65')
('FAIRS19194-1', 'BC857C', '-65')
('FAIRS19194-1',

In [19]:
# corpus = session.query(Corpus).filter(Corpus.name == 'Hardware').one()

# for doc in corpus.documents:
#     if doc.name == 'PNJIS00254-1':
#         d = doc
#         break
# print d

In [20]:
# for phrase in d.phrases:
#     if '55' in phrase.words:
#         p = phrase
#         print p.cell
#         import pdb; pdb.set_trace()

In [21]:
# candies = sorted(candidates, key=lambda x: x[0])

In [22]:
# from hardware_utils import count_hardware_labels

# filename = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
# %time count_hardware_labels(candidates, filename, attrib='stg_temp_min', attrib_class='temp')

## TEMPORARY - Return to Normalcy

In [23]:
# If necessary
import os
os.system('cp snorkel.db snorkel.db\ candidates');

Next, in Part 3, we will load `Labels` for each of our `Candidates` so that we can evaluate performance.